In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer


In [2]:
data = pd.read_excel("../data/ev_final.xlsx")
data.head()

,uid,name,vendor_name,address,latitude,longitude,city,country,open,close,...,postal_code,zone,0,available,capacity,cost_per_unit,power_type,total,type,vehicle_type
0,STATIC12,GensolCharge Pvt. Ltd.,GensolCharge Pvt. Ltd.,"NDSE Grid, BRPL South Extension",28.568238,77.219666,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,2.0,BEVC DC 001,['4W']
1,STATIC14,REIL,REIL,Scada office kalka ji,28.541995,77.260583,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,3.3 kW,NaN,AC,3.0,BEVC AC 001,"['2W', '3W', '4W']"
2,STATIC15,REIL,REIL,Ashram Chowk Mathura Road,28.571189,77.259806,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,2.0,BEVC DC 001,['4W']
3,STATIC16,REIL,REIL,Nizamuddin Railway station,28.588991,77.253240,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,4.0,BEVC DC 001,['4W']
4,STATIC17,BluSmart,BluSmart,"BSES Bhawan, Nehru Place, New Delhi 110048",28.549427,77.254636,Delhi,India,00:00:00,23:59:59,...,110001,central-delhi,NaN,NaN,15 kW,NaN,DC,1.0,BEVC DC 001,['4W']


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2705 entries, 0 to 2704
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   uid              2705 non-null   object 
 1   name             2705 non-null   object 
 2   vendor_name      2705 non-null   object 
 3   address          2705 non-null   object 
 4   latitude         2705 non-null   float64
 5   longitude        2705 non-null   float64
 6   city             2705 non-null   object 
 7   country          2705 non-null   object 
 8   open             2705 non-null   object 
 9   close            2705 non-null   object 
 10  logo_url         2238 non-null   object 
 11  staff            2705 non-null   object 
 12  payment_modes    2705 non-null   object 
 13  contact_numbers  2705 non-null   object 
 14  station_type     2705 non-null   object 
 15  postal_code      2705 non-null   int64  
 16  zone             2410 non-null   object 
 17  0             

In [4]:
data.isnull().sum()

uid                   0
name                  0
vendor_name           0
address               0
latitude              0
longitude             0
city                  0
country               0
open                  0
close                 0
logo_url            467
staff                 0
payment_modes         0
contact_numbers       0
station_type          0
postal_code           0
zone                295
0                  2705
available           238
capacity            208
cost_per_unit       252
power_type          208
total               208
type                208
vehicle_type        208
dtype: int64

In [5]:
data.drop(columns=["uid","name","address","logo_url","staff","payment_modes","contact_numbers",0],inplace=True)

In [6]:
data.isnull().sum()


vendor_name        0
latitude           0
longitude          0
city               0
country            0
open               0
close              0
station_type       0
postal_code        0
zone             295
available        238
capacity         208
cost_per_unit    252
power_type       208
total            208
type             208
vehicle_type     208
dtype: int64

In [7]:

data_raw = data.copy()       # keep original safe
data = data_raw.copy()       # work on this

In [8]:
print(data.shape)
print(data.isnull().sum().sort_values(ascending=False))


(2705, 17)
zone             295
cost_per_unit    252
available        238
capacity         208
vehicle_type     208
total            208
type             208
power_type       208
vendor_name        0
postal_code        0
station_type       0
close              0
open               0
country            0
city               0
longitude          0
latitude           0
dtype: int64


In [9]:
data['capacity_num'] = data['capacity'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
data['capacity_num'] = pd.to_numeric(data['capacity_num'], errors='coerce')


In [10]:
data['cpu_num']=data['cost_per_unit'].astype(str).str.extract(r'(\d+\.?\d*)',expand=False)
data['cpu_num']=pd.to_numeric(data['cost_per_unit'],errors='coerce')


In [11]:
# group-level median by power_type (better than global)
data['capacity_num'] = data.groupby('power_type')['capacity_num'].transform(
    lambda x: x.fillna(x.median())
)
# fallback
data['capacity_num'].fillna(data['capacity_num'].median(), inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_3132\465576841.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['capacity_num'].fillna(data['capacity_num'].median(), inplace=True)


In [12]:
data['cpu_num'] = data.groupby('power_type')['cpu_num'].transform(
    lambda x: x.fillna(x.median())
)
data['cpu_num'].fillna(data.groupby('zone')['cpu_num'].transform('median'), inplace=True)
data['cpu_num'].fillna(data['cpu_num'].median(), inplace=True)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Temp\ipykernel_3132\818878910.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inpl

In [13]:
data['available'] = data.groupby('vendor_name')['available'].transform(
    lambda x: x.fillna(x.median())
)
data['available'].fillna(0, inplace=True)   # conservative default


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: M

In [14]:
data['total'] = data.groupby('vendor_name')['total'].transform(lambda x: x.fillna(x.median()))
data['total'].fillna(data['total'].median(), inplace=True)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\user\AppData\Local\Temp\ipykernel_3132\180288300.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['total'].fillna(data['total'].median(), inplace=True)


In [15]:
zone_mode = data.groupby('city')['zone'].agg(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan).to_dict()
data['zone'] = data.apply(lambda r: zone_mode.get(r['city'], r['zone']) if pd.isna(r['zone']) else r['zone'], axis=1)
data['zone'].fillna('Unknown', inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_3132\1350995127.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['zone'].fillna('Unknown', inplace=True)



### 📘 Zone Imputation Code Explained
---

### 🔹 What It Does

* Finds the most common **zone** for each **city**.
* Fills missing `zone` values with that city’s most common zone.
* If still missing, replaces with `"Unknown"`.

---

### 🔹 Line by Line

**1️⃣ Find mode per city**

* `groupby('city')['zone']` → group rows by city.
* `.agg(lambda s: s.mode().iloc[0])` → take most frequent zone.
* `.to_dict()` → make dictionary → `{'Delhi':'East', 'Mumbai':'West'}`.

**2️⃣ Fill missing values**

* `apply(..., axis=1)` → work row by row.
* If `zone` is missing → look up city’s mode in dictionary.
* If present → keep as is.

**3️⃣ Replace leftovers**

* `.fillna('Unknown')` → any remaining NaN becomes `"Unknown"`.

---

### 🔹 Analogy

Each **city = school**, each **zone = favorite sport**.

* Step 1: Find the most common sport in each school.
* Step 2: Missing student’s sport = school’s most common.
* Step 3: If unknown, mark `"Unknown"`.




In [16]:
# clean to list
data['vehicle_type_clean'] = (
    data['vehicle_type'].astype(str)
        .str.replace(r"[\[\]'\"\s]", "", regex=True)  # remove brackets/quotes/spaces
        .str.split(',')
)

data['supports_4w'] = data['vehicle_type_clean'].apply(lambda x: '4W' in x if isinstance(x, list) else False)
data['supports_2w'] = data['vehicle_type_clean'].apply(lambda x: '2W' in x if isinstance(x, list) else False)
data['n_vehicle_types'] = data['vehicle_type_clean'].apply(lambda x: len(x) if isinstance(x, list) else 0)


🔹 What It Does

Cleans the vehicle_type column into a list.

Creates 3 new features:

supports_4w → True if "4W" present.

supports_2w → True if "2W" present.

n_vehicle_types → Count of vehicle types.

🔹 Line by Line

1️⃣ Clean to list

.astype(str) → ensure values are strings.

.str.replace(r"[\[\]'\"\s]", "", regex=True) → remove [ ] ' " and spaces.

.str.split(',') → split string into list.

Example: "['2W','4W']" → ['2W','4W'].

2️⃣ Supports 4W

lambda x: '4W' in x → check if list contains "4W".

3️⃣ Supports 2W

Same as above, but for "2W".

4️⃣ Count types

len(x) → number of items in list.

🔹 Quick Analogy

Think of vehicle_type as a messy string list:

Step 1 → Clean it up into a proper list.

Step 2 → Check if it supports 4-wheeler.

Step 3 → Check if it supports 2-wheeler.

Step 4 → Count how many types are supported.

In [17]:

num_cols = [col for col in ['capacity_num','cpu_num','available','total'] if col in data.columns]
num_cols_clean = [col for col in num_cols if not data[col].isna().all()]

imputer = KNNImputer(n_neighbors=5)
data[num_cols_clean] = imputer.fit_transform(data[num_cols_clean])



In [18]:
data

,vendor_name,latitude,longitude,city,country,open,close,station_type,postal_code,zone,...,power_type,total,type,vehicle_type,capacity_num,cpu_num,vehicle_type_clean,supports_4w,supports_2w,n_vehicle_types
0,GensolCharge Pvt. Ltd.,28.568238,77.219666,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,2.0,BEVC DC 001,['4W'],15.0,NaN,[4W],True,False,1
1,REIL,28.541995,77.260583,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,AC,3.0,BEVC AC 001,"['2W', '3W', '4W']",3.3,NaN,"[2W, 3W, 4W]",True,True,3
2,REIL,28.571189,77.259806,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,2.0,BEVC DC 001,['4W'],15.0,NaN,[4W],True,False,1
3,REIL,28.588991,77.253240,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,4.0,BEVC DC 001,['4W'],15.0,NaN,[4W],True,False,1
4,BluSmart,28.549427,77.254636,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,1.0,BEVC DC 001,['4W'],15.0,NaN,[4W],True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,1.0,DC-001,['4W'],15.0,NaN,[4W],True,False,1
2701,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,2.0,CCS (DC),['4W'],142.0,NaN,[4W],True,False,1
2702,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,2.0,CHAdeMO (DC),['4W'],142.0,NaN,[4W],True,False,1
2703,Verdemobility,28.572632,77.169680,New Delhi,India,00:00:00,23:59:59,charging,110022,south-west-delhi,...,AC,2.0,TYPE - 2 (AC),['4W'],7.4,NaN,[4W],True,False,1


In [19]:
# Step 1 — re-extract numeric part from cost_per_unit and convert to numeric
data['cpu_num'] = data['cost_per_unit'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
data['cpu_num'] = pd.to_numeric(data['cpu_num'], errors='coerce')

# Quick check — how many missing and basic stats
print("cpu_num missing:", data['cpu_num'].isna().sum(), "of", len(data))
print(data['cpu_num'].describe())


cpu_num missing: 252 of 2705
count    2453.000000
mean        4.126608
std         8.224900
min         0.000000
25%         0.000000
50%         0.000000
75%         2.050000
max        33.000000
Name: cpu_num, dtype: float64


In [20]:
# Assuming you have already handled the first step of extracting numeric values
import pandas as pd
from sklearn.impute import KNNImputer

# Create a copy to compare methods
data_median = data.copy()
data_knn = data.copy()

# Median Imputation
data_median['cpu_num'].fillna(data_median['cpu_num'].median(), inplace=True)
print("Median Imputation:")
print(data_median['cpu_num'].describe())

# KNN Imputation
# Ensure your data is numeric and free of other missing values before using KNN
imputer = KNNImputer(n_neighbors=5)
data_knn['cpu_num'] = imputer.fit_transform(data_knn[['cpu_num']])
print("\nKNN Imputation:")
print(data_knn['cpu_num'].describe())

Median Imputation:
count    2705.000000
mean        3.742170
std         7.923609
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        33.000000
Name: cpu_num, dtype: float64

KNN Imputation:
count    2705.000000
mean        4.126608
std         7.832268
min         0.000000
25%         0.000000
50%         0.000000
75%         4.126608
max        33.000000
Name: cpu_num, dtype: float64


C:\Users\user\AppData\Local\Temp\ipykernel_3132\994366553.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_median['cpu_num'].fillna(data_median['cpu_num'].median(), inplace=True)


In [21]:
data_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2705 entries, 0 to 2704
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   vendor_name         2705 non-null   object 
 1   latitude            2705 non-null   float64
 2   longitude           2705 non-null   float64
 3   city                2705 non-null   object 
 4   country             2705 non-null   object 
 5   open                2705 non-null   object 
 6   close               2705 non-null   object 
 7   station_type        2705 non-null   object 
 8   postal_code         2705 non-null   int64  
 9   zone                2705 non-null   object 
 10  available           2705 non-null   float64
 11  capacity            2497 non-null   object 
 12  cost_per_unit       2453 non-null   object 
 13  power_type          2497 non-null   object 
 14  total               2705 non-null   float64
 15  type                2497 non-null   object 
 16  vehicl

In [22]:
data_knn

,vendor_name,latitude,longitude,city,country,open,close,station_type,postal_code,zone,...,power_type,total,type,vehicle_type,capacity_num,cpu_num,vehicle_type_clean,supports_4w,supports_2w,n_vehicle_types
0,GensolCharge Pvt. Ltd.,28.568238,77.219666,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,2.0,BEVC DC 001,['4W'],15.0,4.126608,[4W],True,False,1
1,REIL,28.541995,77.260583,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,AC,3.0,BEVC AC 001,"['2W', '3W', '4W']",3.3,4.126608,"[2W, 3W, 4W]",True,True,3
2,REIL,28.571189,77.259806,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,2.0,BEVC DC 001,['4W'],15.0,4.126608,[4W],True,False,1
3,REIL,28.588991,77.253240,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,4.0,BEVC DC 001,['4W'],15.0,4.126608,[4W],True,False,1
4,BluSmart,28.549427,77.254636,Delhi,India,00:00:00,23:59:59,charging,110001,central-delhi,...,DC,1.0,BEVC DC 001,['4W'],15.0,4.126608,[4W],True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,1.0,DC-001,['4W'],15.0,12.930000,[4W],True,False,1
2701,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,2.0,CCS (DC),['4W'],142.0,20.140000,[4W],True,False,1
2702,EESL,13.078481,80.249480,Chennai,India,00:00,23:59,charging,600031,Unknown,...,DC,2.0,CHAdeMO (DC),['4W'],142.0,20.140000,[4W],True,False,1
2703,Verdemobility,28.572632,77.169680,New Delhi,India,00:00:00,23:59:59,charging,110022,south-west-delhi,...,AC,2.0,TYPE - 2 (AC),['4W'],7.4,16.000000,[4W],True,False,1


In [23]:
# Assume 'data' is your pandas DataFrame after KNN imputation on cpu_num
# and other imputation steps for columns like 'capacity'.

def clean_dataframe(data_knn):
    # Create a list of columns to drop
    columns_to_drop = [
        'city', 'country', 'open', 'close', 'zone',
        'cost_per_unit', 'vehicle_type_clean', 'postal_code',
        'vehicle_type', 'type'
    ]
    
    # Drop the specified columns
    data_cleaned = data.drop(columns=columns_to_drop, axis=1)

    # Perform One-Hot Encoding on remaining categorical columns
    categorical_columns = ['vendor_name', 'station_type', 'power_type']
    data_encoded = pd.get_dummies(data_cleaned, columns=categorical_columns, drop_first=True)

    # Note: 'drop_first=True' helps avoid multicollinearity by dropping the first category column.
    # It's a good practice, especially for models like linear regression.

    # You can now see the result of the cleaning
    print("Cleaned DataFrame Info:")
    data_encoded.info()
    
    return data_encoded

# Call the function to clean your dataset
# data_cleaned_final = clean_dataframe(data)

In [24]:
cleaned_dataframe=clean_dataframe(data_knn)

Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2705 entries, 0 to 2704
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   latitude                            2705 non-null   float64
 1   longitude                           2705 non-null   float64
 2   available                           2705 non-null   float64
 3   capacity                            2497 non-null   object 
 4   total                               2705 non-null   float64
 5   capacity_num                        2705 non-null   float64
 6   cpu_num                             2453 non-null   float64
 7   supports_4w                         2705 non-null   bool   
 8   supports_2w                         2705 non-null   bool   
 9   n_vehicle_types                     2705 non-null   int64  
 10  vendor_name_BSES                    2705 non-null   bool   
 11  vendor_name_Battery

In [25]:
# 'data_encoded' is your cleaned DataFrame from the last step

def finalize_imputation(df):
    # Step 1: Impute missing cpu_num values
    print("Imputing cpu_num...")
    imputer_cpu = KNNImputer(n_neighbors=5)
    df['cpu_num'] = imputer_cpu.fit_transform(df[['cpu_num']])
    print("cpu_num imputation complete.")

    # Step 2: Clean and impute capacity column
    print("Cleaning and imputing capacity...")
    # Extract numeric part, converting "NaN" strings to actual NaN
    df['capacity'] = df['capacity'].astype(str).str.extract(r'(\d+)').astype(float)
    
    # Impute missing numeric capacity values using KNN
    imputer_capacity = KNNImputer(n_neighbors=5)
    df['capacity'] = imputer_capacity.fit_transform(df[['capacity']])
    
    # Check for remaining missing values
    print("\nFinal Missing Value Count:")
    print(df.isnull().sum())
    
    return df

# Run the function on your DataFrame
# data_final = finalize_imputation(data_encoded)

In [26]:
data_final = finalize_imputation(cleaned_dataframe)

Imputing cpu_num...
cpu_num imputation complete.
Cleaning and imputing capacity...

Final Missing Value Count:
latitude                              0
longitude                             0
available                             0
capacity                              0
total                                 0
capacity_num                          0
cpu_num                               0
supports_4w                           0
supports_2w                           0
n_vehicle_types                       0
vendor_name_BSES                      0
vendor_name_BatterySmart              0
vendor_name_BluSmart                  0
vendor_name_E-Fill Electric           0
vendor_name_EEE                       0
vendor_name_EESL                      0
vendor_name_ElectriVa                 0
vendor_name_GensolCharge Pvt. Ltd.    0
vendor_name_HPCL                      0
vendor_name_JBM Renewables            0
vendor_name_Jio-bp                    0
vendor_name_PlugNgo                   0
vendor_na

In [27]:
data_final.to_csv('../data/cleaned_data.csv', index=False)